In [1]:
!pip install agentpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.9/778.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.3/144.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.3 MB/s eta 0:00:00


In [2]:
import agentpy as ap
import numpy as np
import random, json
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import seaborn as sns, IPython
from matplotlib import pyplot as plt, cm

In [3]:
'''
Grid cell types

gr grass
us street heading up
ds street heading down
rs street heading east
ls street heading left
ic intersection
cw crosswalk
sw sidewalk
'''
gr = -100;
cw, ic, us, ls, ds, rs, sw = -2, -1, 0, 1, 2, 3, 4 # Values are clockwise
traffic_light, car = 100, 101

In [4]:
class CarAgent(ap.Agent):

    """
    Function to setup the car agents
    """
    def setup(self):
      self.env = self.model.env
      self.random = self.model.random
      self.moved = False

      # 0: up, 1: left, 2: down, 3: right
      # This dictionary maps which directions are available to cars traveling in a certain direction
      self.directions = {
          rs: [0, 2, 3],
          us: [0, 1, 3],
          ds: [1, 2, 3],
          ls: [0, 1, 2]
      }

      self.direction = -1

    """
    Function to choose a new direction to go after the next intersection
    """
    def choose_direction(self):
      tile = self.env.get_tile(self.get_position())
      possibleDirections = self.directions[tile]
      self.direction = self.random.choice(possibleDirections)

    """
    Function to run at each step of the simulation
    """
    def execute(self):
      self.moved = False  # Keep track of the agent's movement in this step

      # Check if the car is allowed to move to the next position
      next_position = self.get_next_position()
      if next_position is not None and next_position not in self.env.positions.values():
        if self.env.get_tile(next_position) != cw:
          self.env.move_to(self, next_position)
          self.moved = True

    """
    Function to get the next position for the car depending on the road it's on
    """
    def get_next_position(self):
      position = self.get_position()
      tile = self.env.get_tile(position)
      if tile == rs:
        return (position[0], position[1]+1)
      elif tile == ls:
        return (position[0], position[1]-1)
      elif tile == us:
        return (position[0]-1, position[1])
      elif tile == ds:
        return (position[0]+1, position[1])
      else:
        return None

    """
    Function to get the current position in the grid of the car
    """
    def get_position(self):
      return self.env.positions[self]


In [5]:
class TrafficLightAgent(ap.Agent):
    """
    Setup traffic light agent variables
    """
    def setup(self):
        # Actions are linked to a movement in the grid.
        self.actions = {'up': (-1,0), 'down': (1, 0), 'left': (0, -1), 'right': (0, 1)}
        self.next_positions = {'up':'left', 'left':'down', 'down':'right', 'right':'up'} # Code does not suppoer mixing them around yet
        self.env = self.model.env
        self.p = self.model.p

        # Calculated directions cars will exit from
        self.exit_directions = []

        # Traffic light variables
        self.counter = self.p.traffic_light_step_duration
        self.intersection_position = 'down' # which direction of the street the traffic light is letting go through FIXME: traffic light may not start in the upper left cell


    """
    Function to calculate the exits of each side of the intersection
    """
    def find_exits(self):
        topLeftPos = self.env.positions[self]

        leftExit = (topLeftPos[0], topLeftPos[1]-2)
        rightExit = (topLeftPos[0]+1, topLeftPos[1]+3)
        downExit = (topLeftPos[0]+3, topLeftPos[1])
        upExit = (topLeftPos[0]-2, topLeftPos[1]+1)

        self.exit_directions = [upExit, leftExit, downExit, rightExit]

    """
    Function for each step of the simulation
    """
    def execute(self):

        # Traffic light time counter
        if self.counter <= 0:
          self.start_next_cycle()

        # Check if a car has a green light and is ready to move
        next_car_position = self.next_car_position()
        if self.is_car_in_position(next_car_position):
          # Get agent in 'position next_car_position'
          carAgent = list(self.env.grid[next_car_position][0])[0]
          if not carAgent.moved:
            # Ask agent which direction it wants to go (up, left, down, up)
            # TODO: Check if there is no pedestrian in cross walk
            # Move agent accordingly (provided there are no obstructions)
            exit = self.exit_directions[carAgent.direction]
            if exit not in self.env.positions.values():
              self.env.move_to(carAgent, exit)
              carAgent.choose_direction()

          print("Letting car pass through")

        self.counter -= 1

    """
    Get the position of the car it should be letting through
    """
    def next_car_position(self):
        next_car_orientation = self.actions[self.intersection_position]
        dx, dy = -2 * next_car_orientation[0], -2 * next_car_orientation[1]

        x, y = self.get_position()
        return x + dx, y + dy


    # FIXME: Maybe move to model or grid object
    """
    Get position of agent in environment
    """
    def is_car_in_position(self, pos):
        N, M = self.model.p.street.shape
        # FIXME: Once there are pedestrians, we must differentiate the agent type to only allow cars
        if pos in self.env.positions.values():
          return True

    """
    Get position of agent in environment
    """
    def get_position(self):
        return self.env.positions[self]

    """
    Move agent to next position and reset counter
    """
    def start_next_cycle(self):
        # FIXME: Check what is the next position (this assumes counter clock wise rotation)
        self.env.move_by(self, self.actions[self.intersection_position])
        self.intersection_position = self.next_positions[self.intersection_position]
        self.counter = self.p.traffic_light_step_duration

In [6]:
class StreetGrid(ap.Grid):
    """
    Setup the street tiles
    """
    def setup(self):
        # Initialize the environment
        self.environment = np.copy(self.p.street)

    """
    Get the value of a specified position
    """
    def get_tile(self, pos):
      if 0 <= pos[0] < self.shape[0] and 0 <= pos[1] < self.shape[1]:
        return self.environment[pos]
      else:
        return None

In [9]:
class StreetModel(ap.Model):
    def setup(self):
        self.random.seed = self.p.seed
        self.env = StreetGrid(self, shape=street.shape, torus=True)
        self.traffic_light_agents = ap.AgentList(self, 1, TrafficLightAgent) # FIXME make the same as traffic_light_agents length
        self.cars = ap.AgentList(self, 5, CarAgent)
        self.env.add_agents(self.traffic_light_agents, positions=self.p.traffic_light_agents)
        self.env.add_agents(self.cars, positions=[(6, 0), (6, 2), (0, 5), (5, 11), (11, 6)])
        self.cars.choose_direction()
        self.traffic_light_agents.find_exits()



    def step(self):
        self.cars.execute()
        self.traffic_light_agents.execute()

    # Report found route and Q-values
    def end(self):
        self.report('Smily face', ':)') # FIXME

In [10]:
def animation_plot(model, ax):
    N, M = model.p.street.shape
    grid = np.copy(street)

    # FIXME: allow multiple traffic lights
    traffic_light_agent = list(model.traffic_light_agents)[0]
    traffic_light_state = model.env.positions[traffic_light_agent]
    grid[traffic_light_state] = traffic_light
    carAgents = list(model.cars)

    i = 0
    for carAgent in carAgents:
      grid[carAgent.get_position()] = car+i
      i+=1

    directionArrows = ["⬆︎", "⬅︎", "⬇︎", "➡︎"]
    color_dict = {gr:'#2c7d29', cw: '#ffffff', ic: '#ff0000', us: '#777777', ls: '#777777', ds:'#777777', rs:'#777777', car: "#1e152a", car+1:"#4e6766", car+2:"#5ab1bb", car+3: "#a5c882", car+4: "#f7dd72", traffic_light:'#00ff00', sw: '#a1a7b5'}
    directionDict = {us: 0, ls: 1, ds: 2, rs: 3}

    ap.gridplot(grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title("Intersection Position: {}, Steps left: {}\nLetting car from {} pass through\n".format(traffic_light_agent.intersection_position, traffic_light_agent.counter, traffic_light_agent.next_car_position()))

    for i in range(N):
      for j in range(M):
        tile = model.env.get_tile((i, j))
        if tile in directionDict and (i,j) not in model.env.positions.values():
          ax.text(j, i, directionArrows[directionDict[tile]], color = "#b0b0b0")

    for carAgent in carAgents:
      pos = carAgent.get_position()
      ax.text(pos[1], pos[0], directionArrows[carAgent.direction], color="white")

# # Environment representation with a grid
street =  np.array([
    [gr, gr, gr, gr, sw, ds, us, sw, gr, gr, gr, gr],
    [gr, gr, gr, gr, sw, ds, us, sw, gr, gr, gr, gr],
    [gr, gr, gr, gr, sw, ds, us, sw, gr, gr, gr, gr],
    [gr, gr, gr, gr, sw, ds, us, sw, gr, gr, gr, gr],
    [sw, sw, sw, sw, sw, cw, cw, sw, sw, sw, sw, sw],
    [ls, ls, ls, ls, cw, ic, ic, cw, ls, ls, ls, ls],
    [rs, rs, rs, rs, cw, ic, ic, cw, rs, rs, rs, rs],
    [sw, sw, sw, sw, sw, cw, cw, sw, sw, sw, sw, sw],
    [gr, gr, gr, gr, sw, ds, us, sw, gr, gr, gr, gr],
    [gr, gr, gr, gr, sw, ds, us, sw, gr, gr, gr, gr],
    [gr, gr, gr, gr, sw, ds, us, sw, gr, gr, gr, gr],
    [gr, gr, gr, gr, sw, ds, us, sw, gr, gr, gr, gr]
])

parameters = {
    'street': street,
    'traffic_light_agents': [(5, 5)],  # It is assumed the agent spawns in an 'ic' (intersection) locaiton
    'traffic_light_step_duration': 4,  # FIXME Change to 'per agent' variable
    'cars_agents': [(1,2), (0,2)],    # FIXME Remove (cars should spawn from houses that do not exists yet)
    'steps': 40,
    'seed': 10
}

fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111)
model = StreetModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml())

Letting car pass through
Letting car pass through
Letting car pass through
Letting car pass through
Letting car pass through
Letting car pass through
Letting car pass through
Letting car pass through
Letting car pass through
Letting car pass through
Letting car pass through
Letting car pass through
Letting car pass through
Letting car pass through
Letting car pass through
Letting car pass through
Letting car pass through
Letting car pass through
Letting car pass through
